<a href="https://colab.research.google.com/github/carogaltier/outpatient-demographic-data/blob/main/Hospital_Outpatient_Attendance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demographic Data Processing for Hospital Attendances

This notebook provides a framework for processing demographic data related to hospital outpatient attendances and non-attendances, focusing on gender and age distributions. The data is sourced from the "Outpatient attendances by age and gender 2023-24" table from the Hospital Episode Statistics (HES), NHS England. The reference data is accessible [here](https://digital.nhs.uk/data-and-information/publications/statistical/hospital-outpatient-activity/2023-24).

### Overview

The purpose of this analysis is to convert raw attendance data into a normalized format that can be used to explore patterns in healthcare utilization. By transforming the data into proportional values, this approach enables more effective comparisons across different demographic groups and can inform resource allocation, policy-making, and healthcare service improvements.

The main steps include:

1. **Data Preparation:** Load and clean the data by filtering out pediatric patients (those under 15 years old) to focus on adult hospital attendance patterns.
2. **Normalization:** Convert attendance and non-attendance counts into percentages.
3. **Validation:** Ensure that the resulting proportions are accurate and consistent.
4. **Export:** Save the processed data in a format suitable for future use.


In [8]:
import pandas as pd

# Load the data from the Excel file
demographics_url = "https://files.digital.nhs.uk/34/18846B/hosp-epis-stat-outp-rep-tabs-2023-24-tab.xlsx"
demographics = pd.read_excel(demographics_url, sheet_name="Summary Report 3", header=5, nrows=20)

demographics.head()

,Age (yrs),Attended - Female Maternity,Attended - Female Non-Maternity,Attended - Male,Did Not Attend - All Female,Did Not Attend - Male
0,0-4,99749,1323379,1808686,144448,182248
1,5-9,12,1141459,1410307,123398,153197
2,10-14,1001,1332122,1474473,128988,144733
3,15-19,164129,1560589,1317693,204316,178511
4,20-24,796325,1504401,950748,280326,169559


In [9]:
# Rename columns to lowercase and filter out patients under 15 years old
demographics.columns = [
    'age_yrs',
    'attended_female_maternity',
    'attended_female_non_maternity',
    'attended_male',
    'dna_female',
    'dna_male'
]

# Remove patients under 15 years old and create an explicit copy of the filtered DataFrame
demographics = demographics.loc[~demographics['age_yrs'].isin(['0-4', '5-9', '10-14'])].copy()

# Calculate total female attendance (sum of maternity and non-maternity) and the grand total
demographics['attended_female'] = demographics['attended_female_maternity'] + demographics['attended_female_non_maternity']
grand_total = demographics[['attended_female', 'attended_male', 'dna_female', 'dna_male']].sum().sum()

# Normalize percentages directly
for col in ['attended_female', 'attended_male', 'dna_female', 'dna_male']:
    demographics[col] /= grand_total

# Round the individual columns to 5 decimal places
for col in ['attended_female', 'attended_male', 'dna_female', 'dna_male']:
    demographics[col] = demographics[col].round(5)

# Calculate total percentages for female and male after rounding
demographics['total_female'] = demographics['attended_female'] + demographics['dna_female']
demographics['total_male'] = demographics['attended_male'] + demographics['dna_male']

# Re-round total_female and total_male to 5 decimal places to avoid tiny precision errors
demographics['total_female'] = demographics['total_female'].round(5)
demographics['total_male'] = demographics['total_male'].round(5)

# Validate that the sum of total_female and total_male is exactly 1
total_sum = demographics['total_female'].sum() + demographics['total_male'].sum()
assert round(total_sum, 5) == 1.0, f"Total sum is not 1, it is {total_sum}"

# Select and create the final DataFrame
final_df = demographics[['age_yrs', 'attended_female', 'attended_male', 'dna_female', 'dna_male', 'total_female', 'total_male']]

final_df.head()

,age_yrs,attended_female,attended_male,dna_female,dna_male,total_female,total_male
3,15-19,0.01699,0.01298,0.00201,0.00176,0.01900,0.01474
4,20-24,0.02267,0.00937,0.00276,0.00167,0.02543,0.01104
5,25-29,0.03966,0.01128,0.00394,0.00193,0.04360,0.01321
6,30-34,0.05199,0.01363,0.00447,0.00220,0.05646,0.01583
7,35-39,0.04389,0.01513,0.00388,0.00227,0.04777,0.01740


In [10]:
# Export the final DataFrame to a CSV file
from google.colab import files

final_df.to_csv('attendances_by_age_gender_2023-2024.csv', index=False, float_format='%.5f')
files.download('attendances_by_age_gender_2023-2024.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>